In [5]:
# Originator: Camille
import os
import pandas as pd
current_path = os.getcwd()

# Loading Everyone's data

In [6]:
# Loading Beema's data
b_path = os.path.join(os.getcwd(), "../../raw_data/beema/cleaned_data/")
csv_files = [file for file in os.listdir(b_path) if file.endswith(".csv")]
dataframes = {}
for file in csv_files:
    file_path = os.path.join(b_path, file)
    df = pd.read_csv(file_path)
    dataframes[file] = df
    print(f"Loaded '{file}' with {df.shape[0]} rows and {df.shape[1]} columns.")

Loaded 'billboard_200_cleaned.csv' with 14400 rows and 21 columns.
Loaded 'country_airplay_cleaned.csv' with 14754 rows and 13 columns.
Loaded 'country_on-demand_streaming_cleaned.csv' with 14400 rows and 17 columns.
Loaded 'current_country_albums_cleaned.csv' with 5412 rows and 14 columns.
Loaded 'current_pop_albums_cleaned.csv' with 7247 rows and 14 columns.
Loaded 'current_r&b_hip_hop_albums_cleaned.csv' with 14279 rows and 14 columns.
Loaded 'current_rap_albums_cleaned.csv' with 7303 rows and 14 columns.
Loaded 'current_rock_albums_cleaned.csv' with 3602 rows and 14 columns.
Loaded 'dance_electronic_on-demand_cleaned.csv' with 14400 rows and 17 columns.
Loaded 'heatseekers_cleaned.csv' with 10800 rows and 17 columns.
Loaded 'pop_on-demand_streaming_cleaned.csv' with 14400 rows and 17 columns.
Loaded 'r&b_hip_hop_airplay_cleaned.csv' with 14702 rows and 13 columns.
Loaded 'r&b_hip_hop_albums_cleaned.csv' with 202 rows and 14 columns.
Loaded 'r&b_hip_hop_on-demand_streaming_cleaned.c

In [18]:
# Loading Camille's data
c_path = os.path.join(os.getcwd(), "../../raw_data/camille/")
csv_files2 = [file for file in os.listdir(c_path) if file.endswith(".csv")]
sentiment_dataframes = {}
for file in csv_files2:
    file_path = os.path.join(c_path, file)
    df = pd.read_csv(file_path)
    sentiment_dataframes[file] = df

    print(f"Loaded '{file}' with {df.shape[0]} rows and {df.shape[1]} columns.")

Loaded 'songs_with_attributes_and_lyrics.csv' with 955320 rows and 17 columns.
Loaded 'spotify_aggregated_by_countries.csv' with 72 rows and 7 columns.
Loaded 'top_spotify_songs.csv' with 7477 rows and 25 columns.
Loaded 'universal_top_spotify_songs.csv' with 1475430 rows and 25 columns.


In [13]:
# Loading Shivam's data
s_path = os.path.join(current_path, "../../raw_data/shivam/raw_data/nodes.csv")
nodes_df = pd.read_csv(s_path)
print(nodes_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156422 entries, 0 to 156421
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   spotify_id  156422 non-null  object 
 1   name        156418 non-null  object 
 2   followers   156418 non-null  float64
 3   popularity  156422 non-null  int64  
 4   genres      156422 non-null  object 
 5   chart_hits  19641 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.2+ MB
None


In [14]:
#Process data for sentiment analysis
sen_df = sentiment_dataframes['universal_top_spotify_songs.csv']
sen_df['artists_clean'] = sen_df['artists'].str.lower().str.strip()
nodes_df['name_clean'] = nodes_df['name'].str.lower().str.strip()
merged_df = pd.merge(sen_df,nodes_df,left_on='artists_clean',right_on='name_clean',how='inner')

In [15]:
# Simplify the data
# 1. Remove unnecessary columns
columns_to_drop = ["spotify_id_x", "is_explicit", "duration_ms", "time_signature","artists_clean", "spotify_id_y", "name_y", "name_clean"]
processed_df = merged_df.drop(columns=columns_to_drop)

columns_to_rename = {
    "name_x": "songs",
    "popularity_x": "song_popularity",
    "popularity_y": "artist_popularity"
}
processed_df = processed_df.rename(columns=columns_to_rename)

print("Processed DataFrame Info:")
print(processed_df.info())

Processed DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944165 entries, 0 to 944164
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   songs               944053 non-null  object 
 1   artists             944053 non-null  object 
 2   daily_rank          944165 non-null  int64  
 3   daily_movement      944165 non-null  int64  
 4   weekly_movement     944165 non-null  int64  
 5   country             929704 non-null  object 
 6   snapshot_date       944165 non-null  object 
 7   song_popularity     944165 non-null  int64  
 8   album_name          943407 non-null  object 
 9   album_release_date  943569 non-null  object 
 10  danceability        944165 non-null  float64
 11  energy              944165 non-null  float64
 12  key                 944165 non-null  int64  
 13  loudness            944165 non-null  float64
 14  mode                944165 non-null  int64  
 15  speechin

In [16]:
#2. Remove duplicate columns, keeping only the most recently updated data
processed_df['snapshot_date'] = pd.to_datetime(processed_df['snapshot_date'], errors='coerce')
deduplicated_df = processed_df.sort_values('snapshot_date').drop_duplicates(subset=['songs', 'artists'], keep='last')

print("Deduplicated DataFrame Info:")
print(deduplicated_df.info())

Deduplicated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 7477 entries, 940086 to 0
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   songs               7476 non-null   object        
 1   artists             7476 non-null   object        
 2   daily_rank          7477 non-null   int64         
 3   daily_movement      7477 non-null   int64         
 4   weekly_movement     7477 non-null   int64         
 5   country             7474 non-null   object        
 6   snapshot_date       7477 non-null   datetime64[ns]
 7   song_popularity     7477 non-null   int64         
 8   album_name          7471 non-null   object        
 9   album_release_date  7472 non-null   object        
 10  danceability        7477 non-null   float64       
 11  energy              7477 non-null   float64       
 12  key                 7477 non-null   int64         
 13  loudness            74

In [17]:
# Save the data for analysis to enable repeated use
c_path1 = os.path.join(os.getcwd(), "../../raw_data/camille/")
deduplicated_df.to_csv(os.path.join(c_path1,"top_spotify_songs.csv"), index=False)

In [19]:
lyrics_df = sentiment_dataframes['songs_with_attributes_and_lyrics.csv']
lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955320 entries, 0 to 955319
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                955320 non-null  object 
 1   name              955309 non-null  object 
 2   album_name        385557 non-null  object 
 3   artists           955318 non-null  object 
 4   danceability      955320 non-null  float64
 5   energy            955320 non-null  float64
 6   key               955320 non-null  object 
 7   loudness          955320 non-null  float64
 8   mode              955320 non-null  object 
 9   speechiness       955320 non-null  float64
 10  acousticness      955320 non-null  float64
 11  instrumentalness  955320 non-null  float64
 12  liveness          955320 non-null  float64
 13  valence           955320 non-null  float64
 14  tempo             955320 non-null  float64
 15  duration_ms       955320 non-null  float64
 16  lyrics            95

In [21]:
lyrics_df2 = lyrics_df[['name','artists','lyrics']]

In [23]:
lyrics = pd.merge(deduplicated_df,lyrics_df2,left_on=['songs','artists'],right_on=['name','artists'],how='inner')
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   songs               1438 non-null   object        
 1   artists             1438 non-null   object        
 2   daily_rank          1440 non-null   int64         
 3   daily_movement      1440 non-null   int64         
 4   weekly_movement     1440 non-null   int64         
 5   country             1440 non-null   object        
 6   snapshot_date       1440 non-null   datetime64[ns]
 7   song_popularity     1440 non-null   int64         
 8   album_name          1437 non-null   object        
 9   album_release_date  1437 non-null   object        
 10  danceability        1440 non-null   float64       
 11  energy              1440 non-null   float64       
 12  key                 1440 non-null   int64         
 13  loudness            1440 non-null   float64     

In [24]:
lyrics.to_csv(os.path.join(c_path1,"top_spotify_songs_lyrics.csv"), index=False)